In [ ]:
import numpy as np``

In [ ]:
nbx = 3
nby = 2

In [ ]:
n_x = np.array([2,2,1])
m_y = np.array([3,2])

In [ ]:
alpha_x_y = np.array([[-2,-1],
                      [-3,-2],
                      [-1,-1]])

gamma_x_y = np.array([[10,8],
                      [7,12],
                      [15,9]])

In [ ]:
phi_x_y = alpha_x_y + gamma_x_y
phi_x_y

array([[ 8,  7],
       [ 4, 10],
       [14,  8]])

In [ ]:
pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 90.9 MB/s eta 0:00:00


In [ ]:
import gurobipy as grb

In [ ]:
A1 = np.kron(np.eye(nbx),np.ones((1,nby)))
A1

array([[1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1.]])

In [ ]:
A2=np.kron(np.ones((1,nbx)), np.eye(nby))
A2

array([[1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1.]])

In [ ]:
phi_x_y

array([[ 8,  7],
       [ 4, 10],
       [14,  8]])

In [ ]:
phi_x_y.flatten()

array([ 8,  7,  4, 10, 14,  8])

In [ ]:
m = grb.Model()

In [ ]:
mug_xy = m.addMVar(nbx*nby)

In [ ]:
m.setObjective(mug_xy @ phi_x_y.flatten() , sense = grb.GRB.MAXIMIZE )

In [ ]:
constr1 = m.addConstr( A1 @ mug_xy == n_x )

In [ ]:
constr2 = m.addConstr( A2 @ mug_xy == m_y)

In [ ]:
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x9e0271c4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 5 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+01


In [ ]:
mug_xy.X

array([2., 0., 0., 2., 1., 0.])

In [ ]:
muopt_x_y = mug_xy.X.reshape((nbx,-1))
muopt_x_y

array([[2., 0.],
       [0., 2.],
       [1., 0.]])

In [ ]:
u_x = constr1.pi
u_x

array([ 8., 10., 14.])

In [ ]:
v_y = constr2.pi
v_y

array([0., 0.])

In [ ]:
wlow_x_y = gamma_x_y - v_y[None,:]
wlow_x_y

array([[10.,  8.],
       [ 7., 12.],
       [15.,  9.]])

In [ ]:
wup_x_y = u_x[:,None] - alpha_x_y
wup_x_y

array([[10.,  9.],
       [13., 12.],
       [15., 15.]])

In [ ]:
wup_x_y- wlow_x_y

array([[0., 1.],
       [6., 0.],
       [0., 6.]])

Let's define $w_x_y$ as the middle point between these:

In [ ]:
weq_x_y =(wup_x_y + wlow_x_y) / 2
weq_x_y

array([[10. ,  8.5],
       [10. , 12. ],
       [15. , 12. ]])

In [ ]:
ueq_x = (alpha_x_y + weq_x_y).max(axis = 1)
ueq_x

array([ 8., 10., 14.])

In [ ]:
veq_y = (gamma_x_y - weq_x_y).max(axis = 0)
veq_y

array([0., 0.])

In [ ]:
ueq_x[:,None] == (alpha_x_y + weq_x_y)

array([[ True, False],
       [False,  True],
       [ True, False]])

In [ ]:
pip install pysdot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 12.4 MB/s eta 0:00:00


In [ ]:
from pysdot import PowerDiagram
from pysdot.radial_funcs import RadialFuncUnit
from pysdot import OptimalTransport
from pysdot.domain_types import ConvexPolyhedraAssembly
import numpy as np
import random as rd

In [ ]:
rd.seed(777)
nCells = 10

Ys = np.random.uniform(0,1,2*nCells).reshape((-1,2))
vor_dia = PowerDiagram(Ys)
vor_dia.display_jupyter()

<IPython.core.display.Javascript object>

In [ ]:
rel_tol = 1e-3
q_j = np.ones(nCells)/nCells
vtilde_j = np.zeros(nCells)
cont = True
pow_dia = PowerDiagram(Ys,vtilde_j)
while cont:
    demand_j = pow_dia.integrals()
    if ((demand_j - q_j)/q_j).max()<rel_tol:
        cont=False
    else:
        vtilde_j = vtilde_j - 0.05 * (demand_j - q_j)
        pow_dia.set_weights(vtilde_j)

pow_dia.display_jupyter()

<IPython.core.display.Javascript object>

In [ ]:
def make_square(box=[0, 0, 1, 1]):
    density = ConvexPolyhedraAssembly()
    density.add_box([box[0], box[1]], [box[2], box[3]])
    return density

dens = make_square()

def optimal_transport(density, Y, masses, psi0 = None, err=1e-8):
    center = (density.min_position() + density.max_position())/2
    halfsides = (density.max_position() - density.min_position())/2
    ratio = 1/np.max(np.abs((Y-center)/halfsides))
    psi = (1-ratio)*np.sum((Y-center)**2, axis=-1)
    ot = OptimalTransport(Y, psi, density, radial_func=RadialFuncUnit(), obj_max_dw=err)
    ot.set_masses(masses)
    ot.adjust_weights()
    return ot.pd

pow_dia2 = optimal_transport(dens, Ys, q_j)
vtilde_j2 = pow_dia2.weights
pow_dia2.display_jupyter()

<IPython.core.display.Javascript object>

# Entropic regularization

In [ ]:

from scipy.optimize import minimize

sigma = 1

# Define the function
def f(argument):
  u_x = argument[:nbx]
  v_y = argument[nbx:]
  mu_x_y = np.exp( (phi_x_y - u_x[:,None] - v_y[None,:] -sigma) / sigma )
  return (n_x @ u_x + m_y @ v_y + sigma * mu_x_y.sum())

# Derivative of the function
def gradient(argument):
  u_x = argument[:nbx]
  v_y = argument[nbx:]
  mu_x_y = np.exp( (phi_x_y - u_x[:,None] - v_y[None,:] -sigma) / sigma )
  return np.concatenate([n_x - mu_x_y.sum(axis = 1), m_y - mu_x_y.sum(axis = 0)])


In [ ]:
# Use minimize with the BFGS method
initial_guess = np.zeros(nbx+nby)
#gradient(initial_guess)
result = minimize(f, initial_guess, jac=gradient, method='BFGS')
usol_x = result.x[:nbx]
vsol_y = result.x[nbx:]
print(usol_x-usol_x[-1],vsol_y+usol_x[-1])


[-6.6636506  -7.16533912  0.        ] [13.00020321 15.50199402]


In [ ]:
K_x_y = np.exp( phi_x_y /  sigma -1 )

reltol = 1e-5

B_y = np.ones(nby)
while True:
  A_x = (K_x_y @ (1/B_y)) / n_x
  B_y = (K_x_y.T @ (1/A_x)) / m_y
  if ((K_x_y / (A_x[:,None] * B_y[None,:])).sum(axis = 1) / n_x - 1).max() < reltol:
    break

In [ ]:
usolbis_x = sigma*np.log(A_x)
vsolbis_y = sigma*np.log(B_y)
print(usolbis_x-usolbis_x[-1],vsolbis_y+usolbis_x[-1])

[-6.66364498 -7.16509384  0.        ] [13.00019382 15.5017562 ]


Small value of $\sigma$.

In [ ]:
# Naive try:
sigma = 0.5

K_x_y = np.exp( phi_x_y /  sigma -1 )

reltol = 1e-5
maxiter = 1e3

B_y = np.ones(nby)
t = 0
while True and t < maxiter:
  t += 1
  A_x = (K_x_y @ (1/B_y)) / n_x
  B_y = (K_x_y.T @ (1/A_x)) / m_y
  if ((K_x_y / (A_x[:,None] * B_y[None,:])).sum(axis = 1) / n_x - 1).max() < reltol:
    break

print(t)

1000


## The log-sum-exp trick

In [ ]:
A=2000
B=3

In [ ]:
np.log(np.exp(A)+np.exp(B))

<ipython-input-60-9aeddbc1df0f>:1: RuntimeWarning: overflow encountered in exp
  np.log(np.exp(A)+np.exp(B))


inf

In [ ]:
t = max(A,B)
np.log(np.exp(A-t)+np.exp(B-t))+t

2000.0

In [ ]:
sigma = 0.1

#K_x_y = np.exp( phi_x_y /  sigma -1 )

reltol = 1e-6
maxiter = 1e5

v_y = np.zeros(nby)
t = 0
while True and t < maxiter:
  t += 1
  a_x = (phi_x_y - v_y[None,:]).max(axis = 1) - sigma
  u_x =   a_x + sigma * np.log(   np.exp(   ( phi_x_y - a_x[:,None] - v_y[None,:] - sigma ) /sigma   ).sum(axis=1) / n_x )
  b_y = (phi_x_y - u_x[:,None]).max(axis = 0) - sigma
  vnext_y = b_y + sigma * np.log(  np.exp(  ( phi_x_y - b_y[None,:] - u_x[:,None] - sigma) / sigma   ).sum(axis=0)  / m_y)
  if np.abs((v_y - v_y[0]) - (vnext_y - vnext_y[0])).max() < reltol:
    break
  v_y = vnext_y

print(t)

86785


In [ ]:
print(u_x-u_x[-1],v_y+u_x[-1])

[-6.06931412 -4.2716893   0.        ] [13.9        14.10237458]


In [ ]:
print(usolbis_x-usolbis_x[-1],vsolbis_y+usolbis_x[-1])

[-6.66364498 -7.16509384  0.        ] [13.00019382 15.5017562 ]
